# Install Required Libraries

In [1]:
!pip install ultralytics
!pip install sahi
!pip install opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.3/111.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 9.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=feb0295df114117d4714452eb7f313b9cf7d4f72ac84d44424b368c08dfaa2ad
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.10.0.84
    Uninstalling opencv-python-4.10.0.84:
      Successfully uninstalled opencv-python-4.10.0.84


In [4]:
!pip uninstall -y sahi
!pip install sahi[full]

Found existing installation: sahi 0.11.19
Uninstalling sahi-0.11.19:
  Successfully uninstalled sahi-0.11.19
  Using cached sahi-0.11.19-py3-none-any.whl.metadata (17 kB)
Using cached sahi-0.11.19-py3-none-any.whl (111 kB)


# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Implement Training with Custom Slicing

In [4]:
import os
import cv2
import time
import numpy as np
from ultralytics import YOLO
from tqdm import tqdm

# Function to slice an image into smaller patches
def slice_image(image, slice_height, slice_width, overlap):
    h, w, _ = image.shape
    slices = []
    for y in range(0, h, int(slice_height * (1 - overlap))):
        for x in range(0, w, int(slice_width * (1 - overlap))):
            slice_img = image[y:y + slice_height, x:x + slice_width]
            slices.append((slice_img, (x, y)))
    return slices

# Training
def train_yolov8():
    model = YOLO("/content/drive/MyDrive/hit-uav/yolov8n.pt")  # Load pre-trained YOLOv8 model
    model.train(
        data="/content/drive/MyDrive/hit-uav/hituav.yaml",  # Path to dataset config file
        epochs=100,                    # Number of epochs
        imgsz=640,                     # Image size
        project="/content/HIT-UAV-Model",
        name="yolov8_custom_slicing",
        save=True
    )
    return model

# Run training
yolo_model = train_yolov8()

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/hit-uav/yolov8n.pt, data=/content/drive/MyDrive/hit-uav/hituav.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/HIT-UAV-Model, name=yolov8_custom_slicing2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fa

100%|██████████| 755k/755k [00:00<00:00, 23.4MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 82.2MB/s]


AMP: checks passed ✅


...

    100/100      2.23G      1.042     0.5085      0.894         57        640: 100%|██████████| 126/126 [00:45<00:00,  2.74it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:05<00:00,  1.62it/s]

                   all        287       2460      0.698      0.649      0.701      0.477



100 epochs completed in 1.530 hours.
Optimizer stripped from /content/HIT-UAV-Model/yolov8_custom_slicing2/weights/last.pt, 6.2MB
Optimizer stripped from /content/HIT-UAV-Model/yolov8_custom_slicing2/weights/best.pt, 6.2MB

Validating /content/HIT-UAV-Model/yolov8_custom_slicing2/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:07<00:00,  1.23it/s]


                   all        287       2460       0.66      0.661        0.7      0.482
                person        171       1168      0.909      0.851      0.925       0.51
                   car        136        719      0.953      0.965      0.986      0.766
               bicycle         53        554      0.894      0.823      0.915      0.545
         othervehicles          9         12      0.541      0.667      0.675      0.586
              dontcare          5          7          0          0          0          0
Speed: 0.3ms preprocess, 2.6ms inference, 0.0ms loss, 5.2ms postprocess per image
Results saved to /content/HIT-UAV-Model/yolov8_custom_slicing2


# Files Download and Upload

In [7]:
import os
import shutil

# Define the source and target paths
source_path = "/content/HIT-UAV-Model/yolov8_custom_slicing2/weights/best.pt"
target_directory = "/content/drive/MyDrive/HIT-UAV-Model/"
target_path = os.path.join(target_directory, "best.pt")

# Ensure the target directory exists
os.makedirs(target_directory, exist_ok=True)

# Copy the file
shutil.copy(source_path, target_path)

print(f"Best weights successfully copied to: {target_path}")


Best weights successfully copied to: /content/drive/MyDrive/HIT-UAV-Model/best.pt


In [8]:

# Define the source and target paths
source_path = "/content/HIT-UAV-Model/yolov8_custom_slicing2/weights/last.pt"
target_directory = "/content/drive/MyDrive/HIT-UAV-Model/"
target_path = os.path.join(target_directory, "last.pt")

# Ensure the target directory exists
os.makedirs(target_directory, exist_ok=True)

# Copy the file
shutil.copy(source_path, target_path)

print(f"Last weights successfully copied to: {target_path}")

Last weights successfully copied to: /content/drive/MyDrive/HIT-UAV-Model/last.pt


In [9]:
from google.colab import files

# Download the best.pt file directly
files.download('/content/drive/MyDrive/HIT-UAV-Model/best.pt')
files.download('/content/drive/MyDrive/HIT-UAV-Model/last.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Evaluate the Model

In [11]:
import os
import cv2
import time
import numpy as np
from tqdm import tqdm

def slice_image(image, slice_height, slice_width, overlap):
    h, w, _ = image.shape
    slices = []
    for y in range(0, h, int(slice_height * (1 - overlap))):
        for x in range(0, w, int(slice_width * (1 - overlap))):
            slice_img = image[y:y + slice_height, x:x + slice_width]
            slices.append((slice_img, (x, y)))
    return slices

# Evaluate the model
def evaluate_model(model, dataset_path, slice_size, overlap):
    image_paths = [os.path.join(dataset_path, img) for img in os.listdir(dataset_path) if img.endswith('.jpg')]
    fps_list, inference_times, accuracy_list = [], [], []
    total_start_time = time.time()

    for img_path in tqdm(image_paths):
        image = cv2.imread(img_path)
        slices = slice_image(image, slice_size, slice_size, overlap)
        slice_inference_times = []

        for slice_img, _ in slices:
            start_time = time.time()
            results = model(slice_img)
            slice_inference_times.append(time.time() - start_time)

        inference_times.append(sum(slice_inference_times))
        fps_list.append(len(slices) / sum(slice_inference_times))  # FPS for each image

    total_processing_time = time.time() - total_start_time

    # Metrics
    avg_fps = np.mean(fps_list)
    avg_inference_time = np.mean(inference_times)

    metrics = {
        "FPS": avg_fps,
        "Inference Time (avg per image)": avg_inference_time,
        "Total Processing Time": total_processing_time
    }
    return metrics

# Load the best weights
best_model = YOLO("/content/HIT-UAV-Model/yolov8_custom_slicing2/weights/best.pt")

# Evaluate on the validation dataset
evaluation_metrics = evaluate_model(
    model=best_model,
    dataset_path="/content/drive/MyDrive/hit-uav/images/val",  # Validation images path
    slice_size=640,
    overlap=0.2
)

# Print metrics
print("Evaluation Metrics:")
for key, value in evaluation_metrics.items():
    print(f"{key}: {value:.2f}")

  0%|          | 0/287 [00:00<?, ?it/s]


0: 512x640 24 cars, 1 othervehicles, 39.4ms
Speed: 1.6ms preprocess, 39.4ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)

0: 640x160 12 cars, 34.2ms
Speed: 2.0ms preprocess, 34.2ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 160)


  0%|          | 1/287 [00:00<01:03,  4.50it/s]


0: 512x640 7 persons, 3 cars, 7.1ms
Speed: 2.5ms preprocess, 7.1ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 640)

0: 640x160 (no detections), 6.9ms
Speed: 3.2ms preprocess, 6.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 160)

0: 512x640 3 cars, 16.1ms
Speed: 2.1ms preprocess, 16.1ms inference, 2.1ms postprocess per image at shape (1, 3, 512, 640)

0: 640x160 1 car, 15.0ms
Speed: 1.6ms preprocess, 15.0ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 160)


  1%|          | 3/287 [00:00<00:28,  9.94it/s]

...


0: 512x640 3 cars, 29 bicycles, 14.2ms
Speed: 4.6ms preprocess, 14.2ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)

0: 640x160 (no detections), 19.5ms
Speed: 1.2ms preprocess, 19.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 160)

0: 512x640 11 persons, 2 bicycles, 14.2ms
Speed: 1.7ms preprocess, 14.2ms inference, 4.4ms postprocess per image at shape (1, 3, 512, 640)

0: 640x160 10 persons, 18.7ms
Speed: 1.2ms preprocess, 18.7ms inference, 4.8ms postprocess per image at shape (1, 3, 640, 160)


 99%|█████████▉| 285/287 [00:18<00:00, 14.38it/s]


0: 512x640 2 cars, 10 bicycles, 15.6ms
Speed: 1.5ms preprocess, 15.6ms inference, 4.0ms postprocess per image at shape (1, 3, 512, 640)

0: 640x160 1 car, 16.2ms
Speed: 1.2ms preprocess, 16.2ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 160)

0: 512x640 2 cars, 4 bicycles, 17.2ms
Speed: 1.6ms preprocess, 17.2ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 640)

0: 640x160 1 car, 17.4ms
Speed: 1.1ms preprocess, 17.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 160)


100%|██████████| 287/287 [00:18<00:00, 15.51it/s]

Evaluation Metrics:
FPS: 44.37
Inference Time (avg per image): 0.05
Total Processing Time: 18.51


In [16]:
from ultralytics import YOLO

# Load the trained model
best_model = YOLO("/content/HIT-UAV-Model/yolov8_custom_slicing2/weights/best.pt")  # Adjust path if needed

# Evaluate on the validation dataset
metrics = best_model.val(data="/content/drive/MyDrive/hit-uav/hituav.yaml")  # Path to your dataset YAML file

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/hit-uav/labels/val.cache... 287 images, 0 backgrounds, 0 corrupt: 100%|██████████| 287/287 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:05<00:00,  3.33it/s]


                   all        287       2460      0.659      0.662        0.7      0.478
                person        171       1168      0.908      0.852      0.924      0.511
                   car        136        719      0.952      0.965      0.986      0.766
               bicycle         53        554      0.895      0.827      0.915      0.541
         othervehicles          9         12      0.539      0.667      0.675      0.573
              dontcare          5          7          0          0          0          0
Speed: 0.3ms preprocess, 3.8ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/val4


In [18]:
# Print evaluation metrics
print("Evaluation Metrics:")
print(f"mAP@0.5: {metrics.box.maps[0]:.4f}")  # mAP at IoU threshold 0.5
print(f"mAP@0.5-0.95: {metrics.box.maps[1]:.4f}")  # mAP averaged over IoU thresholds from 0.5 to 0.95
print(f"Precision: {metrics.box.mean_results()[0]:.4f}")  # Mean Precision
print(f"Recall: {metrics.box.mean_results()[1]:.4f}")  # Mean Recall

Evaluation Metrics:
mAP@0.5: 0.5114
mAP@0.5-0.95: 0.7655
Precision: 0.6586
Recall: 0.6621


In [19]:
num_classes = len(metrics.box.maps)
for i in range(num_classes):
    class_precision, class_recall, class_ap50, class_ap = metrics.box.class_result(i)
    print(f"Class {i}: Precision: {class_precision:.4f}, Recall: {class_recall:.4f}, AP@0.5: {class_ap50:.4f}, AP@0.5:0.95: {class_ap:.4f}")

Class 0: Precision: 0.9080, Recall: 0.8519, AP@0.5: 0.9241, AP@0.5:0.95: 0.5114
Class 1: Precision: 0.9520, Recall: 0.9654, AP@0.5: 0.9855, AP@0.5:0.95: 0.7655
Class 2: Precision: 0.8945, Recall: 0.8267, AP@0.5: 0.9151, AP@0.5:0.95: 0.5409
Class 3: Precision: 0.5385, Recall: 0.6667, AP@0.5: 0.6747, AP@0.5:0.95: 0.5726
Class 4: Precision: 0.0000, Recall: 0.0000, AP@0.5: 0.0000, AP@0.5:0.95: 0.0000


In [20]:
# Define the file path where metrics will be saved
output_file_path = "/content/drive/MyDrive/hit-uav/evaluation_summary.txt"

# Open the file in write mode
with open(output_file_path, "w") as f:
    # Write overall evaluation metrics
    f.write("Overall Evaluation Metrics:\n")
    for key, value in evaluation_metrics.items():
        f.write(f"{key}: {value:.2f}\n")

    # Write mAP, Precision, and Recall
    f.write("\nmAP, Precision, and Recall:\n")
    f.write(f"mAP@0.5: {metrics.box.maps[0]:.4f}\n")  # mAP at IoU threshold 0.5
    f.write(f"mAP@0.5-0.95: {metrics.box.maps[1]:.4f}\n")  # mAP averaged over IoU thresholds
    f.write(f"Precision: {metrics.box.mean_results()[0]:.4f}\n")  # Mean Precision
    f.write(f"Recall: {metrics.box.mean_results()[1]:.4f}\n")  # Mean Recall

    # Write per-class metrics
    f.write("\nPer-Class Metrics:\n")
    num_classes = len(metrics.box.maps)
    for i in range(num_classes):
        class_precision, class_recall, class_ap50, class_ap = metrics.box.class_result(i)
        f.write(
            f"Class {i}: Precision: {class_precision:.4f}, Recall: {class_recall:.4f}, "
            f"AP@0.5: {class_ap50:.4f}, AP@0.5:0.95: {class_ap:.4f}\n"
        )

print(f"All evaluation metrics have been saved to: {output_file_path}")

All evaluation metrics have been saved to: /content/drive/MyDrive/hit-uav/evaluation_summary.txt
